In [8]:
import sys
import os
import pandas as pd

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f"{current_date}")

2025-05-10 19:00:00


In [3]:
current_date

Timestamp('2025-05-10 19:00:00')

In [4]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

In [5]:
fetch_data_to

Timestamp('2025-05-10 19:00:00')

In [6]:
fetch_data_from

Timestamp('2025-04-12 19:00:00')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_rides_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_rides_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_rides_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-04.
Loading data for 2024-04...
Total records: 3,217,063
Valid records: 3,207,286
Records dropped: 9,777 (0.30%)
Successfully processed data for 2024-04.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-05.
Loading data for 2024-05...
Total records: 4,230,360
Valid records: 4,215,930
Records dropped: 14,430 (0.34%)
Successfully processed data for 2024-05.
Combining all monthly data...
Data loading and processing complete!


In [11]:
rides

,pickup_datetime,pickup_location_id
613697,2025-04-13 12:38:50.285,2733.03
1662765,2025-04-13 17:22:18.671,2733.03
1474929,2025-04-13 18:07:27.791,2733.03
173330,2025-04-14 08:09:00.946,2733.03
789171,2025-04-14 19:03:45.534,2733.03
...,...,...
2447924,2025-05-03 09:00:32.000,SYS038
2997858,2025-05-03 19:02:21.000,SYS038
2280340,2025-05-06 21:04:19.000,SYS038
2280339,2025-05-07 17:03:17.000,SYS038


In [12]:
rides['pickup_datetime'].describe()

count                          3462826
mean     2025-04-27 06:12:11.297076736
min         2025-04-12 19:00:00.088000
25%      2025-04-19 21:03:33.796999936
50%      2025-04-27 16:11:04.236499968
75%      2025-05-03 21:14:16.750000128
max                2025-05-10 18:59:59
Name: pickup_datetime, dtype: object

In [13]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [14]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-04-12 19:00:00,2733.03,0
1,2025-04-12 20:00:00,2733.03,0
2,2025-04-12 21:00:00,2733.03,0
3,2025-04-12 22:00:00,2733.03,0
4,2025-04-12 23:00:00,2733.03,0
...,...,...,...
1436059,2025-05-10 14:00:00,SYS038,0
1436060,2025-05-10 15:00:00,SYS038,1
1436061,2025-05-10 16:00:00,SYS038,0
1436062,2025-05-10 17:00:00,SYS038,0


In [15]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436064 entries, 0 to 1436063
Data columns (total 3 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   pickup_hour         1436064 non-null  datetime64[ns]
 1   pickup_location_id  1436064 non-null  object        
 2   rides               1436064 non-null  int16         
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 24.7+ MB


In [16]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORK_PROJECT_NAME,
    api_key_value=config.HOPSWORK_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

2025-05-10 15:25:44,125 INFO: Initializing external client
2025-05-10 15:25:44,127 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 15:25:47,081 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680


In [17]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 1436064/1436064 | Elapsed Time: 04:26 | Remaining Time: 00:00


Launching job: citibike_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214680/jobs/named/citibike_hourly_feature_group_1_offline_fg_materialization/executions


(Job('citibike_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)